<a href="https://colab.research.google.com/github/Disciplined-22/gpt_2train_1/blob/main/gpt2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import accelerate
import transformers
from transformers import Trainer

In [ ]:
#function to read text file

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

file_path = "/content/drive/MyDrive/gpt_1/gpt_sub_1/neural.txt"  #The actual file path
text = read_txt(file_path)
print(text)



In [ ]:
#code writes the content of the text variable into a file located in the directory
with open("/content/drive/MyDrive/gpt_1/gpt_sub_1/neural.txt", "w") as f:
    f.write(text)

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [ ]:
#funtion to load data in order to feed data to model

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset



In [ ]:
  # Failed way for using  dataloader from pytorch don't run this code

!pip install datasets
from datasets import load_dataset
dataset = load_dataset('text', data_files= file_path)


from torch.utils.data import DataLoader

def preprocess_function(examples):
    # Encode the text and also return the labels
  return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512, return_tensors='pt')

tokenizer.pad_token = tokenizer.eos_token
  # Tokenize your dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


  # Convert to PyTorch DataLoader
dataloader = DataLoader(tokenized_dataset["train"], batch_size=32, collate_fn=data_collator)


In [ ]:
# Function to load data from a file and prepare it for model input.
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
# This function trains a GPT2 model on a given dataset.
#
# Parameters:
# train_file_path (str): The path to the training data file.
# model_name (str): The name of the pre-trained GPT2 model to use.
# output_dir (str): The directory where the trained model and tokenizer will be saved.
# overwrite_output_dir (bool): If True, overwrite the content of the output directory.
# per_device_train_batch_size (int): The batch size for training.
# num_train_epochs (int): The number of training epochs.
# save_steps (int): The number of steps between each checkpoint save.
def train(train_file_path, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs, save_steps):
  # Load the tokenizer from the pre-trained GPT2 model
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)

  # Load the dataset using the tokenizer
  train_dataset = load_dataset(train_file_path, tokenizer)

  # Load the data collator
  data_collator = load_data_collator(tokenizer)

  # Save the tokenizer to the output directory
  tokenizer.save_pretrained(output_dir)

  # Load the model from the pre-trained GPT2 model
  model = GPT2LMHeadModel.from_pretrained(model_name)

  # Save the model to the output directory
  model.save_pretrained(output_dir)

  # Define the training arguments
  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  # Initialize the trainer with the model, training arguments, data collator, and training dataset
  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  # Train the model
  trainer.train()

  # Save the trained model
  trainer.save_model()


In [ ]:
file_path = file_path
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/gpt_1/gpt_sub_2'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50.0
save_steps = 50000

In [ ]:
# Train
train(
    train_file_path=file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [ ]:
# Function to load a pre-trained GPT2 model from a given path.
# Parameters:
# model_path (str): The path to the pre-trained model.
# Returns:
# model: The loaded GPT2 model.
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

# Function to load a pre-trained GPT2 tokenizer from a given path.
# Parameters:
# tokenizer_path (str): The path to the pre-trained tokenizer.
# Returns:
# tokenizer: The loaded GPT2 tokenizer.
def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

# Function to generate text using a pre-trained GPT2 model.
# Parameters:
# model_path (str): The path to the pre-trained model.
# sequence (str): The initial sequence to start the text generation.
# max_length (int): The maximum length of the generated text.
def generate_text(model_path, sequence, max_length):
    # Load the model and tokenizer from the given path
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)

    # Encode the initial sequence into tokens
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')

    # Generate text from the initial sequence of tokens
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )

    # Decode the generated tokens into text and print it
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))


In [ ]:
model2_path = "/content/drive/MyDrive/gpt_1/gpt_sub_2"
sequence2 = "[Q]what is neural net ?"
max_len = 100
generate_text(model2_path, sequence2, max_len)

[Q]what is neural net?
Neural networks are computational systems that learn from data by manipulating its properties and patterns. Their architecture involves layers that connect nodes to a computer, serving as hubs for connections. The size and complexity of these networks make them suitable for training purposes, such as in games and natural language processing.
Conclusions and Future Directions:
Despite their success, neural nets are not without challenges. Recent advancements, such as attention mechanisms, latent processes, and reinforcement learning,
